In [43]:
import pandas as pd
import os

# 입력 폴더 및 출력 폴더 설정
input_folder = r"C:\Users\ys000\capstone\data\질병_csv"
output_folder = r"C:\Users\ys000\capstone\data\질병_csv\processed"

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 폴더 내 모든 CSV 파일 처리
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):  # CSV 파일만 처리
        input_file = os.path.join(input_folder, filename)

        # CSV 파일 불러오기
        df = pd.read_csv(input_file)

        # 세 번째 행 기준으로 결측값이 아닌 열만 선택
        filtered_df = df.loc[:, df.iloc[1].notnull()]

        # 두 번째 행을 컬럼명으로 설정
        filtered_df.columns = filtered_df.iloc[1]

        # 필요 없는 행 삭제 (상단 2개 행)
        filtered_df = filtered_df[2:].reset_index(drop=True)

        # 파일명에서 확장자 제거
        file_name = df.iloc[5, 0]

        # 첫 번째 컬럼에 '항목' 추가 (파일명 기준)
        filtered_df.insert(0, '항목', file_name)

        # '진료년월' -> '지역'으로 컬럼명 변경
        filtered_df.rename(columns={'진료년월': '지역'}, inplace=True)

        # 불필요한 행 삭제
        filtered_df = filtered_df[1:].reset_index(drop=True)

        # 날짜 형식 컬럼 변환 (2021년06월 -> 2021-06)
        date_columns = [col for col in filtered_df.columns if any(char.isdigit() for char in col)]

        # '연도'와 '환자수' 컬럼 추가
        records = []

        # 각 행을 순회하면서 연도와 환자수를 추출
        for _, row in filtered_df.iterrows():
            항목 = row['항목']
            지역 = row['지역']
            for col in date_columns:
                if pd.notnull(row[col]):
                    # 컬럼명을 YYYY-MM 형식으로 변환
                    year_month = pd.to_datetime(col.replace('년', '-').replace('월', ''), errors='coerce').strftime('%Y-%m')
                    records.append({'항목': 항목, '지역': 지역, '연도': year_month, '환자수': row[col]})

        # 새로운 데이터프레임 생성
        result_df = pd.DataFrame(records)

         # ✅ 저장 파일명을 기후_질병명_건강보험심사평가원.csv 형식으로 지정
        safe_name = str(file_name).replace("/", "_").replace("\\", "_").replace(" ", "")
        output_file = os.path.join(output_folder, f"기후_{safe_name}_건강보험심사평가원.csv")
        
        # 결과 CSV 파일 저장
        result_df.to_csv(output_file, index=False, encoding='utf-8')